Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple,deque
from random import choice
from tqdm.auto import tqdm
from heapq import heappop, heappush
import numpy as np

In [2]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

# Goal state

In [3]:
FINAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
FINAL_STATE

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15,  0]])

In [4]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# BREADTH-FIRST SEARCH (BFS)

In [5]:
def bfs(initial_state: np.ndarray) -> dict:
    visited = set()
    queue = deque([(initial_state, [])])
    steps=0

    while queue:
        current_state, path = queue.popleft()
        state_tuple = tuple(map(tuple, current_state))  #converts the state to something hashable
        
        if state_tuple in visited:
            continue
        visited.add(state_tuple)
        steps+=1

        # Checks if we reached the goal state
        if np.array_equal(current_state, FINAL_STATE):
            print("\nFinal state:")
            print(current_state)
            return {
                'solution': path,
                'quality': len(path),  # number of actions in the solutions
                'cost': steps,         # total number of actions evaluated
                'efficiency': len(path) / steps if steps > 0 else 0  # quality vs cost
            }
        
        # Generates new moves
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            queue.append((new_state, path + [act]))  # adds the new state to the explored states

    return {
        'solution': None,
        'quality': None,
        'cost': steps,
        'efficiency': 0
    }

# A* With Manhattan distance as Heuristic

In [6]:
def manhattan_distance(state: np.ndarray) -> int:
    total_distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue
            target_x, target_y = divmod(value - 1, PUZZLE_DIM)
            total_distance += abs(x - target_x) + abs(y - target_y)
    return total_distance

def astar(initial_state: np.ndarray) -> dict:
    visited = set()
    priority_queue = []
    

    initial_state_tuple = tuple(map(tuple, initial_state))
    heappush(priority_queue, (0, 0, initial_state_tuple, []))
    steps = 0
    
    while priority_queue:
        _, g, current_state_tuple, path = heappop(priority_queue)
        current_state = np.array(current_state_tuple)  # Converti indietro in NumPy array
        
        if current_state_tuple in visited:
            continue
        visited.add(current_state_tuple)
        steps += 1

        # prints the actual state every milion steps just to see what's appening
        if steps % 1000000 == 0:
            print(f"\nExplored states: {steps}")
            print("Current state:")
            print(current_state)
        
        # Checks if we reached the goal state
        if np.array_equal(current_state, FINAL_STATE):
            print("\nFinal state:")
            print(current_state)
            return {
                'solution': path,
                'quality': len(path),  # number of actions in the solutions
                'cost': steps,         # total number of actions evaluated
                'efficiency': len(path) / steps if steps > 0 else 0  # Quality/Cost
            }
        
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_tuple = tuple(map(tuple, new_state))
            if new_state_tuple in visited:
                continue
            new_path = path + [act]
            h = manhattan_distance(new_state)
            f = g + 1 + h
            heappush(priority_queue, (f, g + 1, new_state_tuple, new_path))

    return {
        'solution': None,
        'quality': None,
        'cost': steps,
        'efficiency': 0
    }

In [7]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

print("Initial state:")
print(state)

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Initial state:
[[ 8  5  9  4]
 [ 1 10  2 14]
 [ 7 15 11  6]
 [13  3 12  0]]


In [8]:
# use the commented line to run the bfs algorithm, solution in feasible time only for 3x3 puzzle
#solution = bfs(state)
solution = astar(state)

if solution['solution']:
    print(f"Quality: {solution['quality']}")
    print(f"Cost: {solution['cost']}")
    print(f"Efficiency: {solution['efficiency']:.4f}")
else:
    print("\nNo solution found")


Explored states: 1000000
Current state:
[[ 1  2 14  4]
 [ 7  8  0  6]
 [13  9 10 11]
 [15  5  3 12]]

Final state:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
Quality: 46
Cost: 1937816
Efficiency: 0.0000
